In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import librosa

files_normaly = sorted(glob.glob(os.path.abspath('../input/train/*/normal/*.wav')))
files_anomaly = sorted(glob.glob(os.path.abspath('../input/train/*/anomaly/*.wav')))
files_test = sorted(glob.glob(os.path.abspath('../input/test/*.wav')))

normal = []
for file in files_normaly:
    y, sr = librosa.load(file, sr=None)
    normal.append(y)

normal = np.array(normal)

anomaly = []
for file in files_anomaly:
    y, sr = librosa.load(file, sr=None)
    anomaly.append(y)
anomaly = np.array(anomaly)

test = []
for file in files_test:
    y, sr = librosa.load(file, sr=None)
    test.append(y)
test = np.array(test)


In [ ]:
melspec_normal = []
for n in normal:
    m = librosa.feature.melspectrogram(n, n_mels=256)
    m = librosa.power_to_db(m).astype(np.float32)
    melspec_normal.append(m)
melspec_normal = np.array(melspec_normal)

melspec_anomaly = []
for a in anomaly:
    m = librosa.feature.melspectrogram(a, n_mels=256)
    m = librosa.power_to_db(m).astype(np.float32)
    melspec_anomaly.append(m)
melspec_anomaly = np.array(melspec_anomaly)

melspec_test = []
for t in test:
    m = librosa.feature.melspectrogram(t, n_mels=256)
    m = librosa.power_to_db(m).astype(np.float32)
    melspec_test.append(m)
melspec_test = np.array(melspec_test)



In [ ]:
import librosa.display
librosa.display.specshow(melspec_normal[0])


In [ ]:
train = np.concatenate([melspec_normal, melspec_anomaly])
train = train.reshape(train.shape[0],-1)
test = melspec_test.reshape(melspec_test.shape[0], -1)

target = np.concatenate([np.zeros(len(melspec_normal)), np.ones(len(melspec_anomaly))])

from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train, target, test_size=0.33, random_state=42)


In [ ]:
from sklearn.svm import SVC

clf = SVC()

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_X)
train_X_scale = scaler.transform(train_X)
test_X_scale =scaler.transform(test_X)


In [ ]:
C_range_exp = np.arange(-2.0, 5.0)
C_range = 10 ** C_range_exp

params = [
    {'C': C_range, 'kernel': ['linear','rbf']},
]

gs = GridSearchCV(clf, params, n_jobs=1, verbose=2)
gs.fit(X_train, y_train)


In [7]:
clf.fit(train_X_scale, train_Y)

SVC()

In [8]:
clf.score(test_X, test_Y)

0.48443223443223443

In [9]:
clf.score(test_X_scale, test_Y)

0.9029304029304029

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
clf = RandomForestClassifier()

In [14]:
clf.fit(train_X, train_Y)

RandomForestClassifier()

In [15]:
clf.score(test_X, test_Y)

0.8415750915750916

In [16]:
clf.fit(train_X_scale, train_Y)

RandomForestClassifier()

In [17]:
clf.score(test_X_scale, test_Y)

0.8653846153846154

In [ ]:
pred = clf.predict()